In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../src")

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel("DEBUG")

In [ ]:
import torchChar
import torch.nn as nn
import torch.nn.functional as F

In [5]:
torchChar.get_models_dir() / "alice_v6/checkpoints/epoch=2.ckpot"

AttributeError: module 'torchChar' has no attribute 'get_model_dir'